In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader

import numpy as np
import copy
import time
import os
from tqdm.notebook import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# For reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

Using device: cuda


In [ ]:
cifar10_mean = (0.49139968, 0.48215827, 0.44653124)
cifar10_std = (0.24703233, 0.24348505, 0.26158768)
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean, cifar10_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean, cifar10_std),
])

# Load datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:13<00:00, 12.6MB/s]


In [ ]:
def get_resnet18_for_cifar10(pretrained=True):
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1 if pretrained else None)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    model.maxpool = nn.Identity()
    # Modify final layer
    model.fc = nn.Linear(512, 10)
    return model

In [ ]:
# Training parameters
NUM_EPOCHS = 15
CRITERION = nn.CrossEntropyLoss()

# ResNet-specific parameters
RESNET_LR = 0.01
RESNET_MOMENTUM = 0.9
RESNET_WD = 0.0001

In [ ]:
def train_model(model, optimizer, scheduler, train_loader, test_loader, device, model_name):
    history = {'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': []}
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(NUM_EPOCHS):
        print(f'Epoch {epoch+1}/{NUM_EPOCHS}')
        print('-' * 10)

        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader, desc=f'Training {model_name}', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = CRITERION(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        if scheduler:
            scheduler.step()

        epoch_loss = running_loss / total
        epoch_acc = correct / total
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)

        # Evaluation phase
        model.eval()
        test_loss = 0.0
        test_correct = 0
        test_total = 0

        with torch.no_grad():
            for inputs, labels in tqdm(test_loader, desc=f'Testing {model_name}', leave=False):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = CRITERION(outputs, labels)

                test_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()

        test_epoch_loss = test_loss / test_total
        test_epoch_acc = test_correct / test_total
        history['test_loss'].append(test_epoch_loss)
        history['test_acc'].append(test_epoch_acc)

        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        print(f'Test Loss: {test_epoch_loss:.4f} Acc: {test_epoch_acc:.4f}')

        # Save best model
        if test_epoch_acc > best_acc:
            best_acc = test_epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            print(f"New best accuracy: {best_acc:.4f}")

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model, history

In [ ]:
# Initialize models
resnet18 = get_resnet18_for_cifar10(pretrained=True).to(device)

# ResNet optimizer/scheduler
optimizer_resnet = optim.SGD(resnet18.parameters(), lr=RESNET_LR,
                            momentum=RESNET_MOMENTUM, weight_decay=RESNET_WD)
scheduler_resnet = lr_scheduler.CosineAnnealingLR(optimizer_resnet, T_max=NUM_EPOCHS)

print("Training ResNet-18...")
resnet18, resnet_history = train_model(resnet18, optimizer_resnet, scheduler_resnet,
                                      train_loader, test_loader, device, "ResNet-18")

Training ResNet-18...
Epoch 1/15
----------


Training ResNet-18:   0%|          | 0/782 [00:00<?, ?it/s]

Testing ResNet-18:   0%|          | 0/157 [00:00<?, ?it/s]

Train Loss: 0.8118 Acc: 0.7298
Test Loss: 0.4683 Acc: 0.8439
New best accuracy: 0.8439
Epoch 2/15
----------


Training ResNet-18:   0%|          | 0/782 [00:00<?, ?it/s]

...

Testing ResNet-18:   0%|          | 0/157 [00:00<?, ?it/s]

Train Loss: 0.0245 Acc: 0.9921
Test Loss: 0.1974 Acc: 0.9467
Epoch 15/15
----------


Training ResNet-18:   0%|          | 0/782 [00:00<?, ?it/s]

Testing ResNet-18:   0%|          | 0/157 [00:00<?, ?it/s]

Train Loss: 0.0252 Acc: 0.9922
Test Loss: 0.1933 Acc: 0.9469
